In [1]:
import ipywidgets as w
from IPython.display import display, Javascript
from glob import glob
import vtk
import time, os, errno
import json
import nanohub.remote as nr
from nanohub.uidl.ipywidgets import buildJSX
from js.models import A3DWidget



A3DCard = buildJSX('''
<Material.Card variant="outlined">
  <Material.CardContent>
    <Material.Typography color="textSecondary">
      Job Name
    </Material.Typography>
    <Material.Typography component="h5" variant="h5">
      <span>{job_id:string(0)}</span> - <span>{job_name:string(Untitled)}</span>
    </Material.Typography>
    <Material.Typography color="textSecondary">
      Date
    </Material.Typography>
    <Material.Typography variant="body2" component="p">
      {job_date:string(Untitled)}
    </Material.Typography>  
  </Material.CardContent>
  <Material.CardActions>
    <Material.Button size="small" color="secondary" onClick="[propCall(onClick,'')]">Details</Material.Button>
  </Material.CardActions>
</Material.Card>
''', debugger=False);

HeaderContainer = buildJSX('''
<Material.Card variant="outlined">
    <Material.CardHeader
        title="{job_name:string( )}"
        subheader="{job_date:string( )}"
    >
    <Material.Avatar>H</Material.Avatar>
    </Material.CardHeader>
    <Material.CardContent style="text-align:center">
        <Material.ButtonGroup variant="contained" color="primary" disabled="{job_download:boolean(true)}">
          <Material.Button onClick="[propCall(onDownload,'sim_data_mechanical_input.odb')]">Mechanical ODB</Material.Button>
          <Material.Button onClick="[propCall(onDownload,'sim_data_thermal_input.odb')]">Thermal ODB</Material.Button>
          <Material.Button onClick="[propCall(onDownload,'sim_data_mechanical_input.dat')]">Mechanical Logs</Material.Button>
          <Material.Button onClick="[propCall(onDownload,'sim_data_thermal_input.dat')]">Thermal Logs</Material.Button>
          <Material.Button onClick="[propCall(onRedirect,'')]">Go to project</Material.Button>
        </Material.ButtonGroup>
    </Material.CardContent>
    <Material.CardContent style="text-align:center">
        <Material.Select variant="outlined" value="{job_field:string(TNT11)}" onChange="" disabled="{job_data:boolean(true)}">
            <Material.MenuItem disabled="true" value=" "><div style="font-size:initial">Select a field to Visualize</div></Material.MenuItem>
            <Material.MenuItem value="TNT11" onClick="[stateChange(job_field,'TNT11')]"><div style="font-size:initial">Temperature (NT11)</div></Material.MenuItem>
            <Material.MenuItem value="TFV1" onClick="[stateChange(job_field,'TFV1')]"><div style="font-size:initial">Degree of Bonding (FV1)</div></Material.MenuItem>
            <Material.MenuItem value="TSDV_x" onClick="[stateChange(job_field,'TSDV_x')]"><div style="font-size:initial">Crystallinity (SDV_x)</div></Material.MenuItem>
            <Material.MenuItem disabled="true" value=""><div style="font-size:initial">Stress Components</div></Material.MenuItem>
            <Material.MenuItem value="MS11" onClick="[stateChange(job_field,'MS11')]"><div style="font-size:initial">Stress (S11)</div></Material.MenuItem>
            <Material.MenuItem value="MS22" onClick="[stateChange(job_field,'MS22')]"><div style="font-size:initial">Stress (S22)</div></Material.MenuItem>
            <Material.MenuItem value="MS33" onClick="[stateChange(job_field,'MS33')]"><div style="font-size:initial">Stress (S33)</div></Material.MenuItem>
            <Material.MenuItem value="MS12" onClick="[stateChange(job_field,'MS12')]"><div style="font-size:initial">Stress (S12)</div></Material.MenuItem>
            <Material.MenuItem value="MS13" onClick="[stateChange(job_field,'MS13')]"><div style="font-size:initial">Stress (S13)</div></Material.MenuItem>
            <Material.MenuItem value="MS23" onClick="[stateChange(job_field,'MS23')]"><div style="font-size:initial">Stress (S23)</div></Material.MenuItem>
            <Material.MenuItem disabled="true" value=""><div style="font-size:initial">Spatial Displacement</div></Material.MenuItem>
            <Material.MenuItem value="MU1" onClick="[stateChange(job_field,'MU1')]"><div style="font-size:initial">Displacement (U1)</div></Material.MenuItem>
            <Material.MenuItem value="MU2" onClick="[stateChange(job_field,'MU2')]"><div style="font-size:initial">Displacement (U2)</div></Material.MenuItem>
            <Material.MenuItem value="MU3" onClick="[stateChange(job_field,'MU3')]"><div style="font-size:initial">Displacement (U3)</div></Material.MenuItem>
        </Material.Select>
        <Material.Select variant="outlined" value="{job_plane:string(I)}" onChange="" disabled="{job_data:boolean(true)}">
            <Material.MenuItem disabled="true" value=" "><div style="font-size:initial">Select a plane</div></Material.MenuItem>
            <Material.MenuItem value="I" onClick="[stateChange(job_plane,'I')]"><div style="font-size:initial">X</div></Material.MenuItem>
            <Material.MenuItem value="J" onClick="[stateChange(job_plane,'J')]"><div style="font-size:initial">Y</div></Material.MenuItem>
            <Material.MenuItem value="K" onClick="[stateChange(job_plane,'K')]"><div style="font-size:initial">Z</div></Material.MenuItem>
        </Material.Select>
    </Material.CardContent>
</Material.Card>
''', verbose=False);


In [2]:
class JobDashboard (w.Tab):
    PREFIX = "additive3d_"
    DATA_CARD = "sim_data_card.json"
    MODELMATRIX = (
         1.0,  0.0, 0.0, 0.0,
         0.0,  1.0, 0.0, 0.0,
         0.0,  0.0, 1.0, 0.0,
         0.0,  0.0, 0.0, 1.0
    )
    def __init__(self, project):
        self.container = HeaderContainer()
        self.container.onDownload(lambda obj, buf, s=self : s.download_handler(buf))
        self.container.onRedirect(lambda obj, buf, s=self : s.redirect_handler(buf))
        self.container.observe(lambda v, s=self : s.plane_handler(), "job_plane")
        self.container.observe(lambda v, s=self : s.updateField(), "job_field")
        self.project = project
        self.symlink_force(project['path'], 'current_project')
        self.completed = w.HBox()
        self.running = w.HBox()
        self.error = w.HBox()
        self.pending = w.HBox()
        self.debug = w.Output()
        self.popup = w.Output()
        self.render = A3DWidget(plane="I", filename="sim_data_thermal_input", variable="NT11", samples=13, path="")
        self.render.layout.flex= '1'
        self.render.layout.display="none"

        super(JobDashboard, self).__init__(children=[self.completed, self.running, self.error, self.pending, self.debug])
        self.updateJobs()

    def symlink_force(self, target, link_name):
        try:
            os.symlink(target, link_name)
        except OSError as e:
            if e.errno == errno.EEXIST:
                os.remove(link_name)
                os.symlink(target, link_name)
            else:
                raise e
        
    def download_handler(self, file):
        with self.debug:
            subdir = os.path.join(self.container.job_id, "sim_data")
            url = "https://cdmhub.org/projects/" + self.project['alias']+ "/files/download?"
            link = url + "subdir=" + subdir + "&asset=" + file
        with self.popup:        
            display(Javascript("window.open('%s')" % link))
            
    def redirect_handler(self, file):
        with self.debug:
            subdir = os.path.join(self.container.job_id, "sim_data")
            url = "https://cdmhub.org/projects/" + self.project['alias']+ "/files/browse?"
            link = url + "subdir=" + subdir
        with self.popup:        
            display(Javascript("window.open('%s')" % link))
            
    def click_handler(self, card):
        self.loadJob(card.job_id)
            
    def plane_handler(self):
        with self.debug:
            self.render.plane = self.container.job_plane
            
    def loadJob(self, jobid):
        with self.debug:
            path_file = os.path.join(self.project['path'], jobid, JobDashboard.DATA_CARD)
            if os.path.isfile(path_file):
                path_job = os.path.join(self.project['path'], jobid)
                self.container.job_download = True
                self.container.job_data = True
                f = open(path_file)
                data = json.load(f)
                self.container.job_id = str(jobid)
                self.container.job_name = data["job_name"] + " (" + str(jobid) + ")"
                self.container.job_date = time.ctime(os.path.getctime(path_job))
                self.container.job_status = "completed"
                self.container.job_download = False
                self.container.job_data = False
                self.uploadModel()
                f.close()
                
    def uploadModel(self):
        path_job = os.path.join('current_project', self.container.job_id, "sim_data", "sim_data_mechanical_input_13_K_S33.vti") 
        if os.path.isfile(path_job):
            self.render.layout.display="block"
            subdir = os.path.join('current_project', self.container.job_id, "sim_data")
            self.render.path = subdir + "/"
        else:
            self.render.layout.display="none"

            
    def updateField(self):
            job_field = self.container.job_field
            job_file = ""
            if job_field.startswith("T"):
                job_field = job_field.replace("T","",1)
                self.render.filename = "sim_data_thermal_input"
                self.render.variable = job_field
            else :
                job_field = job_field.replace("M","",1)
                self.render.filename = "sim_data_mechanical_input"
                self.render.variable = job_field


    def updateJobs(self):
        self.set_title(0,"Completed")
        self.set_title(1,"Running")
        self.set_title(2,"Errors")
        self.set_title(3,"Pending")
        main_dir = os.path.join(self.project['path'])
        jobs = []

        for file in os.listdir(main_dir):
            if file.startswith(JobDashboard.PREFIX):
                path_job = os.path.join(self.project['path'],file)
                if os.path.isdir(path_job):
                    path_file = os.path.join(self.project['path'],file, JobDashboard.DATA_CARD)
                    if os.path.isfile(path_file):

                        f = open(path_file)
                        data = json.load(f)
                        
                        status = "running"
                        submit_file = os.path.join(self.project['path'],file, 'sim_data', 'sim_data_thermal.log') 
                        try:
                            open(submit_file).read()
                            thermal_status_file = os.path.join(self.project['path'],file, 'sim_data', 'thermal.stdout') 
                            try:
                                status_lines = open(thermal_status_file).read()
                                if 'COMPLETED' in status_lines:
                                    mechanical_status_file = os.path.join(self.project['path'],file, 'sim_data', 'mechanical.stdout') 
                                    try:
                                        status_lines = open(mechanical_status_file).read()
                                        if 'COMPLETED' in status_lines:
                                            vtk_status_file = os.path.join(self.project['path'],file, 'sim_data', 'sim_data_mechanical_input_13_K_S33.vti') 
                                            try:
                                                status_lines = open(vtk_status_file).read()
                                                status = "completed"
                                            except (FileNotFoundError, FileExistsError) as e:
                                                pass
                                        elif 'exited with errors' in status_lines:
                                            status = "errors"
                                    except (FileNotFoundError, FileExistsError) as e:
                                        pass
                                elif 'exited with errors' in status_lines:
                                    status = "errors"

                            except (FileNotFoundError, FileExistsError) as e:
                                pass
                        except (FileNotFoundError, FileExistsError) as e:
                            status = "pending"

                        jobs.append({
                            "name" : data["job_name"], 
                            "date" : time.ctime(os.path.getctime(path_job)), 
                            "status" : status, 
                            "id" : file
                        })
                        f.close()
        joblist = {"completed":[],"running":[],"errors":[],"pending":[]}
        for j in jobs:
            card = A3DCard(
                job_id=j["id"],
                job_name=j["name"], 
                job_date=j["date"]
            )
            card.onClick(lambda obj, buf, c=card, s=self : s.click_handler(c))
            self.card = card
            joblist[j["status"]].append(card)
            
        self.completed.children = [w.VBox(joblist["completed"]), w.VBox([self.container,self.render], layout=w.Layout(flex="1"))]
        self.running.children = [w.VBox(joblist["running"]), w.VBox([self.container,self.render], layout=w.Layout(flex="1"))]
        self.error.children = [w.VBox(joblist["errors"]), w.VBox([self.container,self.render], layout=w.Layout(flex="1"))]
        self.pending.children = [w.VBox(joblist["pending"]), w.VBox([self.container,self.render], layout=w.Layout(flex="1"))]
        self.set_title(0,"Completed ("+str(len(joblist["completed"]))+")")
        self.set_title(1,"Running ("+str(len(joblist["running"]))+")")
        self.set_title(2,"Errors ("+str(len(joblist["errors"]))+")")
        self.set_title(3,"Pending ("+str(len(joblist["pending"]))+")")
        

In [3]:
class Wizard(w.VBox):
    def __init__(self, **kwargs): 
    
        self.debug = w.Output()
        self.projects = self._set_projects()            
        self.container = w.HBox()
        projects = [(v['title'],k) for k,v in self.projects.items()]
        self.project = w.Dropdown(
            description = "Select Project", 
            style={'description_width': 'initial'}, 
            layout=w.Layout(width='auto'),
            options = projects
        )
        self.project.observe(lambda c, s=self: Wizard._handle_change(s, c), names='value')
        super(Wizard, self).__init__(children=[self.project, self.container, self.debug])

    def _handle_change(self, change):
        with self.debug:
            if self.project.value != "":
                self.container.children = [JobDashboard(self.projects[self.project.value])]
                
    def _set_projects(self):
        projects = {"" : {"title" : "Select a project"}}
        with self.debug:
            auth_data = {
                'grant_type' : 'tool',
            }
            with open(os.environ["SESSIONDIR"]+"/resources") as file:
                lines = [line.split(" ", 1) for line in file.readlines()]
                properties = {line[0].strip(): line[1].strip() for line in lines if len(line)==2}
                auth_data["sessiontoken"] = properties["session_token"]
                auth_data["sessionnum"] = properties["sessionid"]
            session = nr.Session(auth_data, url='https://cdmhub.org/api/')
            req_json = session.requestGet('/projects/list?verbose=1')
            req_json = req_json.json()
            for p in req_json['projects']:
                #if 'a3dconsortium' in p['alias']:
                if 'cams' in p['alias']:
                    path = '/data/projects/' + p['alias'] + "/files"
                    if os.path.exists(path):
                        p['path'] = path
                        projects[p['alias']] = p
        return projects
    


In [4]:
a = Wizard()
a

Wizard(children=(Dropdown(description='Select Project', layout=Layout(width='auto'), options=(('Select a proje…